In [1]:
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.profiler import profile, record_function, ProfilerActivity
from utils import *
from fla_optimized import FLANN, npDataset

In [2]:
X,y,weight = diabetes_data()
X,y = torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
hidden_dims = [50,25,10]
attn_heads = 20
model = FLANN(input_dim=108, hidden_dims=hidden_dims, output_dim=1, attn_heads=attn_heads, activation=nn.ReLU)
criterion = nn.BCEWithLogitsLoss(pos_weight=weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(X.shape[0]/64)

236.703125


In [3]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("back"):
        # model(X[:64])
        loss = criterion(model(X[:64]), y[:64].unsqueeze(1)).backward()        

STAGE:2024-07-23 14:19:41 70719:13408693 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-07-23 14:19:41 70719:13408693 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-07-23 14:19:41 70719:13408693 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [4]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time"))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                        Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------------------------------------------------------------  
                                                   back        12.60%      15.239ms       100.00%     120.947ms     120.947ms             1                                                                  []  
                                          aten::softmax         0.01%       9.000us         6.67%       8.072ms       8.072ms             1                     